In [7]:
import sys

sys.path.append("../")
from library.datasets import ProstateCancerDataset
from library.train import create_dataloader
from entrypoint import prostate_scoring_function
import matplotlib.pyplot as plt
import torch
from library.models import *
import random
import glob
from icecream import ic
import copy
import pprint
from tqdm.auto import tqdm
import pandas as pd

In [2]:
best_prostate_models_combined = [i for i in glob.glob('../tuning_exp_training_details/pretrained_model_raw_metadata/*Combined*/**/*Model.pt', recursive=True)]

In [8]:
prostate_dir = "../datasets/train_prostate/train/"
random_seed = 24042

val_dataset = ProstateCancerDataset(
    prostate_dir,
    split_type="val",
    random_seed=random_seed,
    input_slice_count=3,
)
test_dataset = ProstateCancerDataset(
    prostate_dir, split_type="test", random_seed=random_seed, input_slice_count=3
)

Adding raw case data to ChaimeleonDataset:   0%|          | 0/294 [00:00<?, ?it/s]

Adding raw case data to ChaimeleonDataset:   0%|          | 0/294 [00:00<?, ?it/s]

In [9]:
val_loader = create_dataloader(val_dataset, batch_size=6, shuffle=False)
test_loader = create_dataloader(test_dataset, batch_size=6, shuffle=False)

In [10]:
prostate_model_base = ProstateCombinedResnet18PretrainedModel()

In [11]:
# load model
model_state_dict = torch.load(best_prostate_models_combined[0])
model = copy.deepcopy(prostate_model_base)
model.load_state_dict(model_state_dict)
model = model.eval()

In [12]:
# Prostate models loop
epoch_targets = []
epoch_preds = []
epoch_outputs = []
for batch_idx, (images, metadata, targets) in enumerate(val_loader):
    data = images, metadata.squeeze()
    if isinstance(targets, list):
        pfs = targets[1]
        targets = targets[0]
    else:
        targets = targets
    output = model(data)
    pred = output.argmax(dim=1, keepdim=True)
    
    targets, output, pred = (
            targets.argmax(dim=1, keepdim=True),
            torch.nn.functional.sigmoid(output),
            pred,
        )
    
    epoch_targets.extend([i.item() for i in targets])
    epoch_preds.extend([i.item() for i in pred])
    epoch_outputs.extend(
        [output[k][1].item() for k, i in enumerate(targets)]
    )

score, score_dict = prostate_scoring_function(epoch_targets, epoch_outputs, epoch_preds)

In [13]:
pprint.pprint(best_prostate_models_combined[0])
pprint.pprint(score_dict)

'../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-11-27T22-30-46/[]_frozen/0.01_starting_learning_rate/best_val_score_ProstateCombinedResnet18PretrainedModel.pt'
{'auc_chai': 0.8825757575757575,
 'balanced_accuracy': 0.8825757575757576,
 'roc_auc': 0.9141414141414141,
 'score': 0.8888888888888888,
 'sensitivity': 0.9166666666666666,
 'sensitivity_recall': 0.9166666666666666,
 'specificity': 0.8484848484848485}


In [14]:
def load_saved_model(model_path, model_base):
    model_state_dict = torch.load(model_path)
    model = copy.deepcopy(model_base)
    model.load_state_dict(model_state_dict)
    model = model.eval()

    return model

In [15]:
def evaluate_prostate_model(eval_dataloader, model_path, model_base, verbose=False, eval_id = None):
    model = load_saved_model(model_path, model_base)
    
    epoch_targets = []
    epoch_preds = []
    epoch_outputs = []
    for batch_idx, (images, metadata, targets) in enumerate(eval_dataloader):
        data = images, metadata.squeeze()
        if isinstance(targets, list):
            pfs = targets[1]
            targets = targets[0]
        else:
            targets = targets
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)
        
        targets, output, pred = (
                targets.argmax(dim=1, keepdim=True),
                torch.nn.functional.sigmoid(output),
                pred,
            )
        
        epoch_targets.extend([i.item() for i in targets])
        epoch_preds.extend([i.item() for i in pred])
        epoch_outputs.extend(
            [output[k][1].item() for k, i in enumerate(targets)]
        )
    
    score, score_dict = prostate_scoring_function(epoch_targets, epoch_outputs, epoch_preds)
    if eval_id is not None:
        score_dict = {f"{eval_id}_{key}": val for key, val in score_dict.items()}

    model_details = model_path.split('/')[2:]

    if verbose:
        pprint.pprint(model_details)
        pprint.pprint(score_dict)
        print("\n\n")

    return score_dict

In [16]:
val_results_prostate = {}
for model_path in tqdm(best_prostate_models_combined):
    if 'less_extreme' in model_path:
        results = evaluate_prostate_model(val_loader, model_path, prostate_model_base, eval_id = 'val')
        val_results_prostate[model_path] = results

  0%|          | 0/64 [00:00<?, ?it/s]

In [17]:
test_results_prostate = {}
for model_path in tqdm(best_prostate_models_combined):
    if 'less_extreme' in model_path:
        results = evaluate_prostate_model(test_loader, model_path, prostate_model_base, eval_id = 'test')
        test_results_prostate[model_path] = results

  0%|          | 0/64 [00:00<?, ?it/s]

In [18]:
prostate_val_df = pd.DataFrame.from_dict(val_results_prostate, orient='index')
prostate_test_df = pd.DataFrame.from_dict(test_results_prostate, orient='index')

In [19]:
prostate_val_df

,val_score,val_roc_auc,val_auc_chai,val_balanced_accuracy,val_sensitivity,val_sensitivity_recall,val_specificity
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-17T15-51-14/fixed_augmentation_less_extreme/[]_frozen/0.01_starting_learning_rate_0.75_factor/best_val_score_ProstateCombinedResnet18PretrainedModel.pt,0.775758,0.818182,0.765152,0.765152,0.833333,0.833333,0.696970
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-17T15-51-14/fixed_augmentation_less_extreme/[]_frozen/0.01_starting_learning_rate_0.75_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel.pt,0.741414,0.767677,0.734848,0.734848,0.500000,0.500000,0.969697
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-17T15-51-14/fixed_augmentation_less_extreme/[]_frozen/0.01_starting_learning_rate_0.25_factor/best_val_score_ProstateCombinedResnet18PretrainedModel.pt,0.781313,0.785354,0.780303,0.780303,0.833333,0.833333,0.727273
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-17T15-51-14/fixed_augmentation_less_extreme/[]_frozen/0.01_starting_learning_rate_0.25_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel.pt,0.712121,0.787879,0.693182,0.693182,0.416667,0.416667,0.969697
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-17T15-51-14/fixed_augmentation_less_extreme/[]_frozen/0.003_starting_learning_rate_0.75_factor/best_val_score_ProstateCombinedResnet18PretrainedModel.pt,0.762121,0.871212,0.734848,0.734848,0.500000,0.500000,0.969697
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-17T15-51-14/fixed_augmentation_less_extreme/[]_frozen/0.003_starting_learning_rate_0.75_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel.pt,0.762121,0.871212,0.734848,0.734848,0.500000,0.500000,0.969697
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-17T15-51-14/fixed_augmentation_less_extreme/[]_frozen/0.007_starting_learning_rate_0.5_factor/best_val_score_ProstateCombinedResnet18PretrainedModel.pt,0.743939,0.750000,0.742424,0.742424,0.666667,0.666667,0.818182
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-17T15-51-14/fixed_augmentation_less_extreme/[]_frozen/0.007_starting_learning_rate_0.5_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel.pt,0.635859,0.845960,0.583333,0.583333,0.166667,0.166667,1.000000
"../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-17T15-51-14/fixed_augmentation_less_extreme/['layer2', 'layer3']_frozen/0.01_starting_learning_rate_0.75_factor/best_val_score_ProstateCombinedResnet18PretrainedModel.pt",0.712374,0.758838,0.700758,0.700758,0.916667,0.916667,0.484848
"../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-17T15-51-14/fixed_augmentation_less_extreme/['layer2', 'layer3']_frozen/0.01_starting_learning_rate_0.75_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel.pt",0.628788,0.704545,0.609848,0.609848,0.250000,0.250000,0.969697


In [20]:
prostate_test_df

,test_score,test_roc_auc,test_auc_chai,test_balanced_accuracy,test_sensitivity,test_sensitivity_recall,test_specificity
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-17T15-51-14/fixed_augmentation_less_extreme/[]_frozen/0.01_starting_learning_rate_0.75_factor/best_val_score_ProstateCombinedResnet18PretrainedModel.pt,0.653030,0.734848,0.632576,0.632576,0.416667,0.416667,0.848485
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-17T15-51-14/fixed_augmentation_less_extreme/[]_frozen/0.01_starting_learning_rate_0.75_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel.pt,0.586869,0.722222,0.553030,0.553030,0.166667,0.166667,0.939394
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-17T15-51-14/fixed_augmentation_less_extreme/[]_frozen/0.01_starting_learning_rate_0.25_factor/best_val_score_ProstateCombinedResnet18PretrainedModel.pt,0.678788,0.803030,0.647727,0.647727,0.416667,0.416667,0.878788
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-17T15-51-14/fixed_augmentation_less_extreme/[]_frozen/0.01_starting_learning_rate_0.25_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel.pt,0.584848,0.757576,0.541667,0.541667,0.083333,0.083333,1.000000
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-17T15-51-14/fixed_augmentation_less_extreme/[]_frozen/0.003_starting_learning_rate_0.75_factor/best_val_score_ProstateCombinedResnet18PretrainedModel.pt,0.719192,0.823232,0.693182,0.693182,0.416667,0.416667,0.969697
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-17T15-51-14/fixed_augmentation_less_extreme/[]_frozen/0.003_starting_learning_rate_0.75_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel.pt,0.618182,0.818182,0.568182,0.568182,0.166667,0.166667,0.969697
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-17T15-51-14/fixed_augmentation_less_extreme/[]_frozen/0.007_starting_learning_rate_0.5_factor/best_val_score_ProstateCombinedResnet18PretrainedModel.pt,0.633838,0.684343,0.621212,0.621212,0.333333,0.333333,0.909091
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-17T15-51-14/fixed_augmentation_less_extreme/[]_frozen/0.007_starting_learning_rate_0.5_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel.pt,0.598990,0.828283,0.541667,0.541667,0.083333,0.083333,1.000000
"../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-17T15-51-14/fixed_augmentation_less_extreme/['layer2', 'layer3']_frozen/0.01_starting_learning_rate_0.75_factor/best_val_score_ProstateCombinedResnet18PretrainedModel.pt",0.679040,0.743687,0.662879,0.662879,0.750000,0.750000,0.575758
"../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-17T15-51-14/fixed_augmentation_less_extreme/['layer2', 'layer3']_frozen/0.01_starting_learning_rate_0.75_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel.pt",0.643939,0.719697,0.625000,0.625000,0.250000,0.250000,1.000000


In [21]:
prostate_results_df = pd.merge(prostate_val_df, prostate_test_df, left_index=True, right_index=True)

In [22]:
prostate_results_df.to_csv('./2023_12_16_prostate_eval_results.csv')

In [23]:
prostate_results_df.to_excel('./2023_12_17_prostate_eval_results_w_fixed_less_extreme_test_less_aug.xlsx', sheet_name='prostate_eval_results')